<b>Lab 1 | Week 4 Day 2</b>

First look at Graphs

In [ ]:
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import Annotated
from IPython.display import Image, display

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

import random, gradio as gr

In [ ]:
# Some useful constants

nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["Flying", "Noisy", "Shiny", "Elegant", "Silent", "Smelly", "Slippery", "Soggy", "Slimy", "Bumpy"]

In [ ]:
# Load environment variables
load_dotenv(override=True)

In [ ]:
def shout(text: Annotated[str, "something to be shouted"]) -> str:
    print(text.upper())
    return text.upper()

shout("Hello")

<b> A word about "Annotated" </b>

Type hinting is a feature in Python that lets us specify the type of something:

`my_favorite_things: List`

We can also use something called "Annotated" to add extra information that somebody else might find useful:

`my_favorite_things: Annotated[List, "these are a few of mine"]`

LangGraph needs us to use this feature when we define our State object.

It wants us to tell it what function it should call to update the State with a new value.

This function is called a **reducer**.

LangGraph provides a default reducer called `add_messages` which takes care of the most common case.

That explains why the State looks like this way in Step 1 of the 5 step process.

---


### LangGraph 5 Step Process
<b>Step 1 | Define the State Object</b>

We can use any python object; but it's most common to use a TypedDict or a Pydantic BaseModel.

In [ ]:
class State(BaseModel):
    messages: Annotated[list, add_messages] # add_messages is a reducer

<b>Step 2 | Start the Graph Builder with this State class</b> 

In [ ]:
graph_builder = StateGraph(State) # Just the State class (not State object) is getting passed here

<b>Step 3: Create a Node</b>

A node can be any python function that does something.

The reducer that we set before gets automatically called to combine this response with previous responses


In [ ]:
def my_first_node(state: State) -> State:
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]

    new_state = State(messages=messages)

    return new_state

graph_builder.add_node("first node", my_first_node)

<b>Step 4: Create Edges</b>

We can print and see the grapgh visually

In [ ]:
graph_builder.add_edge(START, "first node")
graph_builder.add_edge("first node", END)

<b>Step 5: Compile Graph</b>

In [ ]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

And now - it's showtime 

In [ ]:
# Define a Gradio chat function

def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]

    state = State(messages=messages)
    result = graph.invoke(state)

    print(result)
    return result["messages"][-1].content

In [ ]:
# Launch chat
gr.ChatInterface(chat, type="messages").launch()

### But what did it tell us?

It makes the point that LangGraph is all about python functions - it doesn't need to involve LLMs!!

---

Now we'll do the 5 steps again with LLM, but in 1 shot:


In [ ]:
# Step 1: Define the State object
class State(BaseModel):
    messages: Annotated[list, add_messages] # add_messages is a reducer

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# Step 3: Create a Node and add it to the graph

llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(state: State) -> State:
    response = llm.invoke(state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# Step 4: Create Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

#### That's it! And, let's do this:

In [ ]:
def chat(user_input: str, hitory):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)

    print(result)
    return result['messages'][-1].content



In [ ]:
gr.ChatInterface(chat, type="messages").launch()